# Cenário 2: 16 qubits e 1 alvo

- Objetivo: Validar o funcionamento em baixa escala ao verificar se o estado alvo aparece com alta probabilidade nas medições. 

Neste cenário, usamos:
- Número de qubits: $n = 2$
- Espaço de busca: $N = 2^n = 4$
- Número de alvos: $M = 1$
- Alvo: $|11\rangle$

O número ideal de iterações de Grover é:
$$
k \approx \left\lfloor \frac{\pi}{4}\sqrt{\frac{N}{M}} \right\rfloor
$$